# Drone Simulator

In [37]:
%matplotlib inline

from py_drone_module import Drone, DroneReference, DroneReferenceFunction, AdaptiveODEOptions
from py_drone_module import eul2quat, quatmultiply, quatconjugate
from DroneParamsAndMatricesBuilder import buildDroneParamsAndMatrices
from numpy import array, diag
from math import pi
import Reference
import DronePlotter
import matplotlib.pyplot as plt
import time

from ipywidgets import interact, interactive
import ipywidgets as widgets

### Attitude and Altitude Controller Settings

In [38]:
Q_att = diag((139.6245112700232,
              139.6245112700232,
              15.2811761590895,
              1.1505204155597211,
              1.1505204155597211,
              0.1209919487616804,
              9.976475759487083e-08,
              9.976475759487083e-08,
              9.976475759487083e-09))
R_att = diag((8, 8, 8))
K_pi_alt = array((0.0013, 0.0747, 0.1068, -0.01))
maxIntegralInfluence = 0.001

### Drone Model and Controllers

In [39]:
dp = buildDroneParamsAndMatrices()
d = Drone(dp)
controller = d.getController(Q_att, R_att, K_pi_alt, maxIntegralInfluence)

### Reference

In [40]:
def get_reference(t):
    m = 0.5  # time scale factor

    qz = eul2quat((10*pi/180, 0, 0))
    qy = eul2quat((0, 10*pi/180, 0))
    qx = eul2quat((0, 0, 10*pi/180))
    q = eul2quat((0, 0, 0))

    if (t >= m * 1 and t < m * 3):
        q = quatmultiply(q, qz)
    if (t >= m * 5 and t < m * 7):
        q = quatmultiply(q, qy)
    if (t >= m * 9 and t < m * 11):
        q = quatmultiply(q, qx)

    if (t >= m * 13 and t < m * 15):
        q = quatmultiply(q, quatconjugate(qy))
    if (t >= m * 17 and t < m * 19):
        q = quatmultiply(q, quatconjugate(qx))
    if (t >= m * 21 and t < m * 23):
        q = quatmultiply(q, quatconjugate(qz))

    if (t >= m * 27 and t < m * 30):
        q = quatmultiply(q, qy)
    if (t >= m * 28 and t < m * 31):
        q = quatmultiply(q, qx)

    if (t >= m * 33 and t < m * 36):
        q = quatmultiply(q, quatconjugate(qx))
    if (t >= m * 34 and t < m * 37):
        q = quatmultiply(q, quatconjugate(qy))

    rr = DroneReference()
    rr.setOrientation(q)
    rr.setPosition((0, 0, 1.0 * (t >= m * 16)))
    return rr.asColVector()

ref_function = DroneReferenceFunction(Reference.get_reference)

### Initial Conditions and Integration Options

In [41]:
x0 = d.getStableState()

odeopt = AdaptiveODEOptions()
odeopt.t_start = 0
odeopt.t_end = 16
odeopt.epsilon = 1e-6
odeopt.h_start = 1e-6
odeopt.h_min = 1e-8
odeopt.maxiter = int(1e6)

### Plotting

In [42]:

def update(mass):
    dp = buildDroneParamsAndMatrices(m=mass)
    # print(dp.uh)
    assert(dp.uh <= 0.90)
    d = Drone(dp)
    start_time = time.time()
    result = d.simulate(controller, ref_function, x0, odeopt)
    end_time = time.time()
    # print("Simulation took {}ms".format((end_time - start_time) * 1000))
    fig, lines = DronePlotter.plot(result, vertical=True, w=1080, h=1920, subsample=2)
    fig.show()

In [43]:
interact(update, 
         mass=widgets.FloatSlider(min=1.0,max=3.0,step=0.1,value=1.8,continuous_update=False));

interactive(children=(FloatSlider(value=1.8, continuous_update=False, description='mass', max=3.0, min=1.0), O…